<a href="https://colab.research.google.com/github/jeffblackadar/image_work/blob/master/coin_photo_processor_v5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Coin photo processing.
This program opens a raw image of a coin, removes the background, crops it and then crops it again to get just to portrait. 
it does just whitespace cropping, 2 coins in column on left side of photo.

## First mount the Google drive:

In [0]:
# Mount the drive first.
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


# Cropping and saving of images

In [0]:
# Working coin image processor - just whitespace cropping, 2 coins in column on left side of photo
import numpy as np
import cv2
import time
from google.colab.patches import cv2_imshow

def img_remove_whitespace(imgo):
    print("start " + str(time.time()))
    #convert to black and white - make it simpler?
    # define a threshold, 128 is the middle of black and white in grey scale
    thresh = 128

    # assign blue channel to zeros
    img_binary = cv2.threshold(imgo, thresh, 255, cv2.THRESH_BINARY)[1]
    #cv2_imshow(img_binary) 
    
    gray_img = cv2.cvtColor(img_binary, cv2.COLOR_BGR2GRAY)
    #cv2_imshow(gray_img) 
    print(gray_img.shape)

    # Thanks https://likegeeks.com/python-image-processing/
    # croppedImage = img[startRow:endRow, startCol:endCol]
      
    # allow for 254 (slightly less than every pixel totally white to allow some specks)
    #count in from the right edge until the mean of each column is less than 255
    for img_row_top in range(0,round(gray_img.shape[0]/2)):    
        if np.mean(gray_img,axis = 1)[img_row_top] < 254:
            break 
    print(img_row_top)
    for img_row_bottom in range(gray_img.shape[0]-1,round(gray_img.shape[0]/2),-1):
        if np.mean(gray_img,axis = 1)[img_row_bottom] < 254:
          break 
    print(img_row_bottom)    
    for img_col_left in range(1,round(gray_img.shape[1]/2)):
        if np.mean(gray_img,axis = 0)[img_col_left] < 254:
            break 
    print(img_col_left)    
    for img_col_right in range(gray_img.shape[1]-1,round(gray_img.shape[1]/2),-1):
        if np.mean(gray_img,axis = 0)[img_col_right] < 254:
            break
    print(img_col_right)
          
    imgo_cropped = imgo[img_row_top:img_row_bottom,img_col_left:img_col_right,0:3]
    print("Whitespace removal")
    print(imgo_cropped.shape)
    
    # cv2_imshow(imgo_cropped) 
    print("end " + str(time.time()))
    return(imgo_cropped)
  
# take second element for sort
def takeThird(elem):
    return elem[2] 
  


def img_remove_background_using_black(imgo):  
    img_remove_background_using_black_success = False
    output = imgo.copy()
    gray_img = cv2.cvtColor(output, cv2.COLOR_BGR2GRAY)
    
    # convert to black and white - make it simpler?
    # define a threshold, 128 is the middle of black and white in grey scale
    thresh = 128

    # assign blue channel to zeros
    img_binary = cv2.threshold(gray_img, thresh, 255, cv2.THRESH_BINARY)[1]
    # cv2_imshow(img_binary) 
    
    # It's more efficient to crop am image of just the coin using whitespace, so try that first.
    # If not successful try to find a circle.
    # A penny is approximately 540 pixels in diameter with this method and settings
    # The photo is not cropped correctly if the radius is too large or small.
    good_coin_radius = 410
    output = img_remove_whitespace(output)
    print("Removed whitespace.")
    print(output.shape)
    # cv2_imshow(output)
    width_half = round(output.shape[0]/2)
    height_half = round(output.shape[1]/2)
    r = width_half
    if (r < good_coin_radius*1.1) and (r > good_coin_radius*0.9):
        #Add a white circle to cover other background elements and coin legend    
        cv2.circle(output,(width_half, height_half), round(r*1.414), (255,255,255), round(r*1.45))
        print("This image looks right.")
        # cv2_imshow(output)   
        img_remove_background_using_black_success = True
        #crop after adding circle
        output = img_remove_whitespace(output)
        return([img_remove_background_using_black_success,output])
    if r > good_coin_radius*1.05:
        print("The image of this coin is too large after background removal.  It may have shadows or debris beyond the coin.")
        cv2_imshow(output)
        return([False])
    if r < good_coin_radius*0.95:
        print("The image of this coin is too small after background removal.  It may be over exposed.")
        cv2_imshow(output)
        return([False])
      
    return([False])
  
  
# https://likegeeks.com/python-image-processing/
import cv2
from google.colab.patches import cv2_imshow
# Set this for IMG_0XXX where XXX = range
for batch_num in range(318,328):
    img = cv2.imread("/content/drive/My Drive/coin-image-processor/photos/raw/IMG_0"+str(batch_num)+".JPG")
    print("/content/drive/My Drive/coin-image-processor/photos/raw/IMG_0"+str(batch_num)+".JPG")

    #Split up the image into 4 images of 1 coin each
    width_half = round(img.shape[0]/2)
    height_half = round(img.shape[1]/2)

    coins = []
    coins.append(img[0:height_half*1, 0:width_half*1])
    coins.append(img[height_half*1:height_half*2, 0:width_half*1])

    for coin_num in range(0,2):
        print("coin_num " + str(coin_num))
        #cv2_imshow(coins[coin_num]) 

        coin1img_no_background = img_remove_background_using_black(coins[coin_num])
        if not coin1img_no_background[0]==False:    
            cv2_imshow(coin1img_no_background[1]) 
            cv2.imwrite("/content/drive/My Drive/coin-image-processor/photos/elizabeth_young/eII"+str(batch_num)+str(coin_num)+".png", coin1img_no_background[1])
        else:
            print("could not get this image.")




